# Fabric Capacity Utilization Monitoring

**Production monitoring solution** that collects capacity metrics and workload data from Microsoft Fabric REST APIs and sends to Azure Log Analytics.

## What This Monitors
- ✅ **Capacity Inventory**: Basic capacity details, workspace counts, item distribution
- ✅ **Workload Data**: Item-based workload tracking across all workspaces
- ✅ **Workspace Events**: Workspace lifecycle and item details

## Data Streams
- `Custom-FabricCapacityMetrics_CL` - Capacity utilization metrics
- `Custom-FabricCapacityWorkloads_CL` - Workload and item inventory
- `Custom-FabricWorkspaceEvents_CL` - Workspace events and changes
- `Custom-FabricItemDetails_CL` - Enhanced item metadata

In [ ]:
# === Framework Integration for Capacity Monitoring ===
import sys
import os

# Add the framework to path  
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Import framework components
from fabricla_connector.config import get_config
from fabricla_connector.api import FabricAPIClient
from fabricla_connector.collectors import CapacityCollector
from fabricla_connector.ingestion import FabricIngestion
from fabricla_connector.utils import create_time_window
from dotenv import load_dotenv

# Load configuration
load_dotenv()
config = get_config()

print("✅ FabricLA Connector framework loaded for capacity monitoring")

In [ ]:
# === Framework-Based Configuration for Capacity Monitoring ===

# Configuration parameters
capacity_ids = [
    # Add your capacity IDs here, e.g.:
    "12345678-1234-1234-1234-123456789abc",
]

# Time window configuration
lookback_hours = 24  # Monitor last 24 hours of capacity usage

# Stream name for capacity data (must match DCR configuration)
stream_capacity = "Custom-FabricCapacityUtilization_CL"

# Validation
workspace_id = config.get("FABRIC_WORKSPACE_ID")
if not workspace_id:
    print("⚠️  Warning: FABRIC_WORKSPACE_ID not set - some features may be limited")

required_vars = ["FABRIC_TENANT_ID", "FABRIC_APP_ID", "AZURE_MONITOR_DCE_ENDPOINT", "AZURE_MONITOR_DCR_IMMUTABLE_ID"]
missing = [var for var in required_vars if not config.get(var)]

if missing:
    print(f"❌ Missing required environment variables: {', '.join(missing)}")
else:
    print("✅ Framework configuration validated")

print(f"\n📊 Capacity Monitoring Configuration:")
print(f"  Capacities to monitor: {len(capacity_ids)}")
print(f"  Lookback period: {lookback_hours} hours")
print(f"  Stream name: {stream_capacity}")
print(f"  DCE Endpoint: {config.get('AZURE_MONITOR_DCE_ENDPOINT', 'Not set')}")

In [ ]:
# === Capacity Utilization Data Collection and Ingestion ===
import datetime as dt
import json

print("🚀 Starting capacity utilization data collection...")
print("=" * 60)

# Initialize framework components
try:
    fabric_client = FabricAPIClient()
    capacity_collector = CapacityCollector(fabric_client)
    
    # Initialize ingestion client
    ingestion_client = FabricIngestion(
        endpoint_host=f"https://{dcr_endpoint_host}",
        dcr_id=dcr_immutable_id,
        stream_name=stream_capacity
    )
    
    print("✅ Framework components initialized successfully")
    
except Exception as e:
    print(f"❌ Framework initialization failed: {e}")
    raise

# Initialize summary for tracking results
summary = {
    "collection_timestamp": dt.datetime.utcnow().isoformat() + "Z",
    "lookback_hours": lookback_hours,
    "workspace_id": workspace_id,
    "configuration": {
        "dcr_endpoint_host": dcr_endpoint_host,
        "dcr_immutable_id": dcr_immutable_id,
        "capacity_items": len(capacity_ids)
    }
}

# === COLLECT CAPACITY UTILIZATION DATA ===
capacity_rows = []

if capacity_ids:
    print(f"\n⚡ Collecting capacity utilization for {len(capacity_ids)} capacities...")
    
    for capacity_id in capacity_ids:
        print(f"   Collecting from capacity: {capacity_id}")
        
        try:
            # Collect capacity metrics
            metrics = capacity_collector.collect_capacity_metrics(
                capacity_id=capacity_id,
                lookback_hours=lookback_hours
            )
            capacity_rows.extend(metrics)
            print(f"     Found {len(metrics)} metric records")
            
        except Exception as e:
            print(f"     ❌ Error collecting from capacity {capacity_id}: {e}")
            
    print(f"✅ Capacity collection completed: {len(capacity_rows)} records")
else:
    print("⏭️  No capacity IDs configured - skipping capacity collection")

# === INGEST DATA TO AZURE MONITOR ===
if capacity_rows:
    print(f"\n📤 Ingesting {len(capacity_rows)} capacity utilization records...")
    
    try:
        capacity_result = ingestion_client.ingest_enhanced(
            records=capacity_rows,
            troubleshoot=True
        )
        
        summary["capacity_metrics"] = {
            "collected": len(capacity_rows),
            "ingested": capacity_result.get("successful_records", 0),
            "failed": capacity_result.get("failed_records", 0),
            "success_rate": capacity_result.get("success_rate", 0)
        }
        
        print(f"     ✅ Capacity metrics: {capacity_result.get('successful_records', 0)}/{len(capacity_rows)} ingested")
        
    except Exception as e:
        print(f"     ❌ Capacity ingestion failed: {e}")
        summary["capacity_metrics"] = {"collected": len(capacity_rows), "ingested": 0, "error": str(e)}
else:
    print("⏭️  No capacity utilization records to ingest")
    summary["capacity_metrics"] = {"collected": 0, "ingested": 0}

# === SUMMARY REPORT ===
total_collected = len(capacity_rows)
total_ingested = summary.get("capacity_metrics", {}).get("ingested", 0)

print("\n" + "=" * 60)
print("✅ CAPACITY UTILIZATION MONITORING COMPLETED")
print("=" * 60)
print(f"📊 Total records collected: {total_collected}")
print(f"📤 Total records ingested: {total_ingested}")
print(f"⚡ Capacity metrics: {total_ingested}")

if total_ingested > 0:
    print(f"\n🎯 Data should appear in Log Analytics within 5-15 minutes")
    print(f"   Table: {stream_capacity} → FabricCapacityUtilization_CL")
else:
    print(f"\n⚠️  No records ingested - check your capacity IDs and configuration")

print(f"\n📄 Detailed summary available in 'summary' variable")
print(json.dumps(summary, indent=2, default=str))

In [ ]:
# === Capacity Monitoring Troubleshooting ===
import json
import datetime as dt

print("🔍 CAPACITY UTILIZATION MONITORING TROUBLESHOOTING")
print("=" * 50)

# 1. Display summary
if 'summary' in locals():
    print("\n1. COLLECTION & INGESTION SUMMARY:")
    print(json.dumps(summary, indent=2, default=str))
    
    total_collected = summary.get("capacity_metrics", {}).get("collected", 0)
    total_ingested = summary.get("capacity_metrics", {}).get("ingested", 0)
    
    print(f"\n📊 Summary Statistics:")
    print(f"   Total collected: {total_collected}")
    print(f"   Total ingested: {total_ingested}")
    print(f"   Success rate: {(total_ingested/total_collected*100) if total_collected > 0 else 0:.1f}%")
    
    if total_ingested == 0:
        print("\n❌ No data was ingested")
        print("🔍 Possible causes:")
        print("   - Invalid capacity IDs")
        print("   - Insufficient permissions to access capacity metrics")
        print("   - No capacity utilization during lookback window")
        print("   - Authentication issues")
    else:
        print(f"\n✅ {total_ingested} capacity utilization records ingested successfully")
else:
    print("❌ No summary data available - run data collection cell first")

# 2. Configuration verification
print("\n2. CONFIGURATION VERIFICATION:")
print(f"   Workspace ID: {workspace_id or 'Not set (some capacity APIs may not require this)'}")
print(f"   Capacity count: {len(capacity_ids)}")
print(f"   Lookback period: {lookback_hours} hours")
print(f"   DCR Endpoint: https://{dcr_endpoint_host}")
print(f"   Stream: {stream_capacity}")

# 3. KQL queries for verification
print("\n3. KQL VERIFICATION QUERIES:")
print("   Run these queries in your Log Analytics workspace:")
print("   ```kql")
print("   // Check capacity utilization table")
print("   FabricCapacityUtilization_CL | count")
print()
print("   // Recent capacity metrics")
print("   FabricCapacityUtilization_CL")
print("   | where TimeGenerated > ago(24h)")
print("   | project TimeGenerated, CapacityId = capacity_id_s, CPUUtilization = cpu_utilization_d, MemoryUtilization = memory_utilization_d")
print("   | order by TimeGenerated desc")
print("   | take 20")
print()
print("   // Capacity utilization trends")
print("   FabricCapacityUtilization_CL")
print("   | where TimeGenerated > ago(7d)")
print("   | summarize")
print("       AvgCPU = avg(cpu_utilization_d),")
print("       MaxCPU = max(cpu_utilization_d),")
print("       AvgMemory = avg(memory_utilization_d),")
print("       MaxMemory = max(memory_utilization_d)")
print("   by capacity_id_s, bin(TimeGenerated, 1h)")
print("   | order by TimeGenerated desc")
print()
print("   // Peak usage identification")
print("   FabricCapacityUtilization_CL")
print("   | where TimeGenerated > ago(24h)")
print("   | where cpu_utilization_d > 80 or memory_utilization_d > 80")
print("   | project TimeGenerated, capacity_id_s, cpu_utilization_d, memory_utilization_d")
print("   | order by cpu_utilization_d desc")
print("   ```")

# 4. Sample data inspection
if 'capacity_rows' in locals() and capacity_rows:
    print("\n4. SAMPLE CAPACITY DATA:")
    sample = capacity_rows[0]
    print("   First capacity utilization record structure:")
    for key, value in list(sample.items())[:8]:  # Show first 8 fields
        print(f"     {key}: {value}")
    if len(sample) > 8:
        print(f"   ... and {len(sample)-8} more fields")

# 5. Capacity monitoring insights
print("\n5. CAPACITY MONITORING INSIGHTS:")
print("   📊 Key metrics to monitor in Log Analytics:")
print("   - CPU and memory utilization trends")
print("   - Peak usage times and resource bottlenecks")
print("   - Capacity scaling requirements")
print("   - Performance correlation with workload patterns")
print("   - Cost optimization opportunities")

print("\n6. CAPACITY OPTIMIZATION RECOMMENDATIONS:")
print("   ⚡ Based on utilization data:")
print("   - Identify optimal capacity sizing")
print("   - Schedule workloads during low-usage periods")
print("   - Monitor for consistent over/under-utilization")
print("   - Plan capacity scaling based on growth trends")
print("   - Optimize workload distribution across capacities")

# 7. Alerting suggestions
print("\n7. SUGGESTED ALERTS:")
print("   🚨 Create Log Analytics alerts for:")
print("   - CPU utilization > 90% for sustained periods")
print("   - Memory utilization > 95%")
print("   - Unusual capacity usage spikes")
print("   - Capacity unavailability or errors")

# 8. Next steps
print("\n8. RECOMMENDED NEXT STEPS:")
if 'summary' in locals() and total_ingested > 0:
    print("   ✅ Data collection successful")
    print("   📊 Build capacity utilization dashboards")
    print("   🚨 Set up alerting for high utilization")
    print("   📈 Analyze usage patterns for optimization")
    print("   💰 Monitor for cost optimization opportunities")
else:
    print("   ❌ Investigate data collection issues:")
    print("   1. Verify capacity IDs are correct and accessible")
    print("   2. Check admin permissions for capacity metrics")
    print("   3. Verify capacity is active and has utilization data")
    print("   4. Test with a larger lookback window")

print(f"\n⏰ Troubleshooting completed at: {dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC")

In [ ]:
# === Advanced Capacity Analytics and Best Practices ===

print("? ADVANCED CAPACITY ANALYTICS")
print("=" * 40)

# Sample advanced analytics queries for capacity optimization
advanced_queries = {
    "Peak Hours Analysis": """
    FabricCapacityUtilization_CL
    | where TimeGenerated > ago(30d)
    | extend Hour = datetime_part('hour', TimeGenerated)
    | summarize AvgCPU = avg(cpu_utilization_d), AvgMemory = avg(memory_utilization_d) by Hour
    | order by AvgCPU desc
    """,
    
    "Capacity Right-sizing": """
    FabricCapacityUtilization_CL
    | where TimeGenerated > ago(7d)
    | summarize 
        P95_CPU = percentile(cpu_utilization_d, 95),
        P95_Memory = percentile(memory_utilization_d, 95),
        Avg_CPU = avg(cpu_utilization_d),
        Avg_Memory = avg(memory_utilization_d)
    by capacity_id_s
    | extend 
        CPURecommendation = case(
            P95_CPU > 90, "Scale Up - High utilization",
            P95_CPU < 30, "Scale Down - Low utilization", 
            "Optimal sizing"
        )
    """,
    
    "Cost Optimization": """
    FabricCapacityUtilization_CL
    | where TimeGenerated > ago(24h)
    | where cpu_utilization_d < 20 and memory_utilization_d < 20
    | summarize LowUtilizationHours = count() by capacity_id_s
    | where LowUtilizationHours > 12
    | project capacity_id_s, LowUtilizationHours, PotentialSavings = "Consider downsizing"
    """,
    
    "Workload Correlation": """
    // Correlate capacity usage with pipeline/dataflow runs
    FabricCapacityUtilization_CL
    | where TimeGenerated > ago(24h)
    | join kind=inner (
        FabricPipelineRun_CL
        | where TimeGenerated > ago(24h)
        | summarize PipelineRuns = count() by bin(TimeGenerated, 1h)
    ) on $left.TimeGenerated
    | project TimeGenerated, cpu_utilization_d, memory_utilization_d, PipelineRuns
    | order by TimeGenerated desc
    """
}

print("🔍 Advanced Analytics Query Examples:")
for title, query in advanced_queries.items():
    print(f"\n📈 {title}:")
    print("```kql")
    print(query.strip())
    print("```")

print("\n💡 CAPACITY MONITORING BEST PRACTICES:")
print("=" * 40)

best_practices = [
    "📊 Monitor both CPU and memory utilization patterns",
    "⏰ Track peak usage times to optimize workload scheduling", 
    "📈 Use percentile analysis (P95, P99) for capacity planning",
    "🔄 Correlate capacity usage with specific workloads",
    "💰 Identify cost optimization opportunities through low utilization periods",
    "🚨 Set up proactive alerts before reaching capacity limits",
    "📋 Regularly review capacity allocation vs actual usage",
    "🎯 Plan capacity scaling based on business growth projections"
]

for practice in best_practices:
    print(f"  {practice}")

print(f"\n🎯 CAPACITY PLANNING GUIDELINES:")
print("=" * 40)
print("  📊 Utilization Targets:")
print("    - CPU: Target 60-80% average, 90% peak maximum")
print("    - Memory: Target 70-85% average, 95% peak maximum")
print("  ")
print("  ⚡ Scaling Triggers:")
print("    - Scale up: >90% utilization for >2 hours")
print("    - Scale down: <30% utilization for >24 hours")
print("  ")
print("  💰 Cost Optimization:")
print("    - Review capacities with <20% utilization")
print("    - Consider auto-scaling for variable workloads")
print("    - Schedule non-critical workloads during off-peak hours")

print(f"\n📋 Regular Capacity Health Checks:")
print("  🔍 Weekly Reviews:")
print("    - Capacity utilization trends")
print("    - Peak usage identification")
print("    - Performance bottleneck analysis")
print("  ")
print("  📈 Monthly Planning:")
print("    - Capacity growth requirements")
print("    - Cost optimization opportunities")
print("    - Workload distribution optimization")

In [ ]:
# Data Collection
print("📊 Starting data collection...")

# Get base data
print("   🏢 Fetching capacities...")
all_capacities = get_capacities(fabric_token)

print("   📁 Fetching workspaces...")
all_workspaces = get_workspaces(fabric_token)

# Filter capacities if specific IDs provided
if capacity_ids:
    target_ids = [cid.lower() for cid in capacity_ids]
    capacities_to_monitor = [cap for cap in all_capacities 
                           if cap.get('id', '').lower() in target_ids]
    print(f"   🎯 Filtered to {len(capacities_to_monitor)} specific capacities")
else:
    capacities_to_monitor = all_capacities
    print(f"   🌐 Monitoring all {len(capacities_to_monitor)} accessible capacities")

# Collect monitoring data
print("   📈 Collecting capacity metrics...")
capacity_metrics = collect_capacity_metrics(capacities_to_monitor, all_workspaces)

print("   💼 Collecting workload data...")
workload_data = collect_workspace_workloads(all_workspaces, fabric_token)

print("   📋 Collecting workspace events...")
workspace_events = collect_workspace_events(all_workspaces, fabric_token)

print("   📦 Collecting item details...")
item_details = collect_item_details(all_workspaces, fabric_token)

# Summary
print(f"\n📊 Collection Summary:")
print(f"   Capacity metrics: {len(capacity_metrics)} records")
print(f"   Workload data: {len(workload_data)} records")
print(f"   Workspace events: {len(workspace_events)} records")
print(f"   Item details: {len(item_details)} records")

print("✅ Data collection completed")

In [ ]:
# Send Data to Log Analytics
print("📤 Sending data to Azure Log Analytics...")

results = {}

# Send capacity metrics
if capacity_metrics:
    print(f"   📈 Sending {len(capacity_metrics)} capacity metrics...")
    result = post_to_log_analytics(dcr_endpoint_host, dcr_immutable_id, 
                                 stream_capacity_metrics, capacity_metrics, monitor_token)
    results["capacity_metrics"] = result
    print(f"      ✅ Sent {result['sent']} records in {result['batches']} batches")

# Send workload data
if workload_data:
    print(f"   💼 Sending {len(workload_data)} workload records...")
    result = post_to_log_analytics(dcr_endpoint_host, dcr_immutable_id, 
                                 stream_capacity_workloads, workload_data, monitor_token)
    results["workload_data"] = result
    print(f"      ✅ Sent {result['sent']} records in {result['batches']} batches")

# Send workspace events (if DCR stream exists)
if workspace_events:
    try:
        print(f"   📋 Sending {len(workspace_events)} workspace events...")
        result = post_to_log_analytics(dcr_endpoint_host, dcr_immutable_id, 
                                     stream_workspace_events, workspace_events, monitor_token)
        results["workspace_events"] = result
        print(f"      ✅ Sent {result['sent']} records in {result['batches']} batches")
    except RuntimeError as e:
        if "not configured" in str(e):
            print(f"      ⚠️  Stream {stream_workspace_events} not configured in DCR - skipping")
        else:
            raise

# Send item details (if DCR stream exists)
if item_details:
    try:
        print(f"   📦 Sending {len(item_details)} item details...")
        result = post_to_log_analytics(dcr_endpoint_host, dcr_immutable_id, 
                                     stream_item_details, item_details, monitor_token)
        results["item_details"] = result
        print(f"      ✅ Sent {result['sent']} records in {result['batches']} batches")
    except RuntimeError as e:
        if "not configured" in str(e):
            print(f"      ⚠️  Stream {stream_item_details} not configured in DCR - skipping")
        else:
            raise

print("\n✅ Data ingestion completed successfully!")
print(f"\n📋 Final Results:")
for stream_type, result in results.items():
    print(f"   {stream_type}: {result['sent']} records sent")

## ✅ Monitoring Complete

### Infrastructure Setup
**Important**: Before running this notebook, deploy the required Azure infrastructure using:
- **Terraform**: `logAnalytics/terraform/` folder
- **Bicep**: `logAnalytics/bicep/` folder

Both create:
- Log Analytics Workspace with custom tables
- Data Collection Endpoint (DCE)
- Data Collection Rules (DCR) with all required streams

### Data Collected
- **Capacity Metrics**: Basic capacity inventory with workspace counts
- **Workload Data**: Item-based workload tracking across workspaces  
- **Workspace Events**: Workspace inventory and item distribution
- **Item Details**: Enhanced metadata for all Fabric items

### Next Steps
1. **Deploy Infrastructure**: Use Terraform or Bicep templates first
2. **Configure Environment**: Set DCR_ENDPOINT_HOST and DCR_IMMUTABLE_ID from deployment outputs
3. **Schedule this notebook** to run regularly (daily/weekly)
4. **Build dashboards** in Azure Monitor/Power BI using the collected data
5. **Set up alerts** based on capacity utilization trends

### KQL Query Examples
```kql
// Capacity utilization overview
FabricCapacityMetrics_CL
| summarize arg_max(TimeGenerated, *) by CapacityId
| project CapacityName, CapacityType, WorkspaceCount, Region

// Item distribution by type
FabricCapacityWorkloads_CL
| summarize count() by WorkloadType, CapacityId
| order by count_ desc

// Workspace activity
FabricWorkspaceEvents_CL
| where TimeGenerated > ago(7d)
| summarize TotalItems = sum(ItemCount) by WorkspaceName
```

### Infrastructure Files
- `logAnalytics/terraform/` - Terraform templates for Azure resources
- `logAnalytics/bicep/` - Bicep templates for Azure resources
- `logAnalytics/common/` - Shared JSON templates for table and DCR definitions